# Interrupts
  + Edge interrupts are triggered on an edge and indicated one or more edges have happened. Level interrupts indicate a persistent condition. In some architectures edge interrupts are cleared automatically when the corresponding ISR is executed, while level triggers are only cleared if the device causing the interrupt is satisfied with the attention given by the CPU. If not, the device will hold the level interrupt and CPU will enter the ISR back again right after returning from the ISR. Interrupts can be classified based on the following methods:
  + Synchronous/Asynchronous - Asynchronous interrupts are typically caused by external interrupts (GPIO), while synchronous originate from software or hardware that is internal to the system
  + FIQs each have their own set of CPU registers, while IRQs don't. So for IRQs, the ISR much save processor context if it plans to use some registers. So it is slower than IRQ. NVIC basically converts the IRQ line that generated the interrupt into a vector address (say, over a 4-bit bus connected to the processor from NVIC) 

## References
  + [Linux Interrupts](https://linux-kernel-labs.github.io/refs/heads/master/lectures/interrupts.html#)
  
# SW vs. HW breakpoints
SW breakpoints are basically instructions at a particular point in RAM replaced with breakpoint instruction. So it is arch specific. GDB replaces the instruction with a trap or illegal divide or some other instruction that iwll cause an exception. SW breakpoints probably, by design, won't work if code is executing off of XIP flash. 

HW breakpoints are registers that can be written to using JTAG. There are comparators inside the processor that constantly compare PC with these registers - Whenever there is a hit, it will stop execution. 

In ARM processors, the FPB unit, i.e., [Flash patch and breakpoints unit](https://developer.arm.com/documentation/ddi0337/h/debug/about-the-flash-patch-and-breakpoint-unit--fpb-) has the comparators needed for hardware breakpoints

**References**
+ [HW vs. SW interrupts](https://stackoverflow.com/questions/8878716/what-is-the-difference-between-hardware-and-software-breakpoints#:~:text=In%20brief%2C%20hardware%20breakpoints%20make,RAM%20memory%20with%20breakpoint%20instruction.)
+ [GDB breakpoint handling](https://sourceware.org/gdb/wiki/Internals/Breakpoint%20Handling)

# Boundary scan vs. JTAG vs. Trace
Boundary scan feature was invented to access pins of an IC soldered on to a PCB. JTAG (Joint Test Action Group) is a standardized and advanced version of boundary scan that allows the user to read/write/toggle any IO pin of any IC soldered on to a PCB. When we have a lot of ICs on a PCB, accessing the pins to check, say integrity of interconnections between two ICs or solder bridges, shorts etc., becomes difficult to do manually or using a test jig. This is particularly so if the ICs have lots of pins, very small pin pitches and if they are ball grid arrays etc. Since JTAG is standardized and is daisy chainable, only can have a single JTAG port on the PCB through which one can directly read/set/reset/continuously toggle any IO pin of any IC in that daisy chain without having to solder test wires or use pogo pins on test points. Later many microcontroller, processor manufacturers expanded JTAG so that a user can also read/write registers in the uC. This enables debugging or testing some SoC functionality without having to use the CPU in an SoC.

ARM Embedded Trace Macrocell (ETM) interface enables you to connect an external ETM unit (for example XDS560V2 Pro) to the processor for real-time code tracing of the core. JTAG signals are also required for ETM trace.

**From a random guy on the internet**:

trace is the record of a instruction sequence what happend before the
trigger point (e.g. a breakpoint). With other words, a trace feature is
usefull if a breakpoint triggers and you like to see what reasons were
therefore and why the CPU arrived at the breakpoint location.

Some JTAG CPUs have seperate connectors (e.g. OCDS Level1 and Level2) to
record the instructions by data and addressbus with external trace memory.
Other embedded CPUs have internal trace fifo
(e.g. Motorolas 9S8) what can be read out over JTAG or BDM or
similar Interfaces even while CPU is running. Some of the new controllers
do not have a test mode what allows to access external bus lines and even
the special bondout chips are very expensive and make many trouble for
emulator manufacturers. The benefit of external trace is the bandwidth
combined with complex trigger conditions but for most problems simple
internal trace memory will do the job why I think this will be the future.

**A different random guy on the internet**:

Departing from PowerPC ARM's trace solution makes use of an Embedded Trace
Macrocells which essentially snoops on the core an outputs a highly
compressed stream which can be reconstructed into an historical record of
the execution of the processor and memory transfers.

This is exceedingly useful for debugging several classes of problems,
particularly ones which do not occur when a conventional debugger is
attached due to the debugger changing the timing characteristics of the
system.

The ETM also includes powerful filtering capabilities to for trace port
bandwidth conservation.

Some PowerPC processors have a system called NEXUS which is based on the
same general idea but most of the implementations I know of aren't as
powerful as the ETM based solution see http://www.nexus5001.org/ for
details.

Since trace is non invasive it can also be used for non invasive profiling
of release code.

# Simulator vs. Emulator
In the context of a microcontroller, a simulator uses a PC to simulate the execution of a code in the microcontroller. Processor read writes to register, memory are simulated in software. But external stimulus is modeled in a file (which has samples of that stimulus). The simulator could be made clock accurate, but not time accurate (it will take the same number of clocks as a mC would, but in terms of time, it could take more or less).

Emulators use ICE/JTAG to basically let you load (modified code) easily into say some SRAM from which it is executed. The user can then use breakpoints, add register to watchlist, read to regs, write to regs (actual mCs regs). 

# FPGA vs CPLD
PLD has logic blocks with fuses interconnecting gates that can be blown (or not blown) to achieve required combinational logic circuit. CPLDs have several PLD blocks, where the connections between the PLDs themselves is configurable. CPLDs can be thought of configurable PCB with with several 74xx series logic ICs. Original PLDs were not reconfigurable and were based on ROMs. Modern ones are reconfigurable - they are based out of EEPROM. Many can be reconfigured several hundred times. CPLDs retain programmed logic through power cycle and will start functioning right after power up.

FPGAs have much finer logic blocks that CPLDs and hence more customizable than CPLDs. They are based on RAMs. FPGAs do not retain their configuration and hence require time to configure themselves using configuration stored in a flash. This means they cannot start working right after power up - like uCs they may take several seconds to boot up and become functional. FPGA can be thought of as reconfigurable silicon cores rather than reconfigurable PCBs. FPGAs may have fixed function blocks such as multipliers, RAMs, PCIE controllers etc.

# EERAM
EERAM = EEPROM + SRAM in one chip with some power loss detection and data backup circuitry. 

**Microchip marketing material**:
EERAM is a serial SRAM product that has hidden nonvolatile bits. On any power disruption, the active SRAM array content is moved to the nonvolatile bits.  When power is restored, the nonvolatile content is restored to the SRAM array and SRAM operation continues

+ Unlimited Reads/Unlimited Writes: Standard SRAM cells and protocol with symmetrical read and write timing
+ Industry-Standard Bus Types: SPI, I2C
+ Invisible-to-User Data Transfers on Power Loss: Hidden nonvolatile cells capture SRAM content; power loss is detected automatically by the EERAM device
+ 100,000 Backups: Data can be moved from SRAM to nonvolatile cells > 100,000 times
+ 100-Year Data Retention: With power removed, data is kept safe for up to 100 years; when power is restored, data is moved back to SRAM for continuing operation
+ Quality: EERAM is based on high-volume, standard CMOS processes to create a very low-cost nonvolatile SRAM solution and the highest quality levels

# Power management
In Tegra, Power and Boot management were put together in one subsystem because when power comes up, the power-on-reset circuit needs to be on the rail that PMIC always turns on by default. And right after power-on-reset hardware, comes booting process. Furthermore, when boot is done, the subsystem would be useless if it isn't doing some power management functions - like DVFS, clock/power gating, turning on/off rails etc. While going into sleep state, the power management has to do required clock/power gating, saving of states etc., in a particular sequence and repeat in the reverse during wake up. But during the sleep, the power management module still needs to be awake to be able to process wake interrupts.

# EABI
Embedded Application Binary Interface is just an ABI with a fancy 'Embedded' attached to it - ARM uses this terminology. What an ABI is that it is the way a compiler achieves interactions between two binaries. For ex., ABI determines how function call is implmented in assembly language (how parameters are passed, how the processor state is saved etc.). It also applies to how function calls are implemented in assembly. ABIs are specific to processors, and possibly, operating systems. For ex., arm-none-eabi toolchain is for ARM based systems when there is no OS. and arm-eabi is typically for ARM based systems with linux.you

# C stuff
## Overview
C code uses system calls (man syscalls in linux) in order to access files/hardware. System calls are OS specific. So for portability, we generally do not use syscalls directly in C code. Instead we use some library functions to make syscalls on our behalf, where the library calls are the same for any OS, and the library functions take care of calling the appropriate system call for a given OS. One examples is the C standard library (*libc*). For ex., we generally use *printf* in our code - In linux, if we run our code executable using *strace* (*strace ./mycode*), it will show all the syscalls made by our code and we can see that *printf* in turn calls the system call "write* which in linux is used for writing into files/pipes/sockets etc. *write* itself is actually a wrapper in *libc* to make a system call - but it is a far more simple than printf.

On a side note, not all library calls are about system calls, but many are. 

The *#include* statements include function declarations for the library functions called in the code. But they don't contain the funciton definitions. They are contained in libc files (or other C libraries) that are automatically used by compilers while compiling our code. For example gcc automatically includes libc. But if we want, we can use system calls directly in our code (if we know that our code will never be executed in a different OS) using 'syscall' assembly code. Then we can compile with the *-nostdlib* option in gcc and need not include any .h file. Note that, the *main()* as the entry point in the code is only valid if we are using libc. In that case, libc adds a boiler plate of code before the actual code and calls our *main()*. If we don't use libc, we need to have a *_start()* instead of *main()*. 

In fact, if we want to do away with pretty much all boiler plate code (some are included by gcc), we can use -nostartfiles. Note that this doesn't mean we aren't using *libc* (-nostdlib is independent of -nostartfiles) - It only means we are not doing the boiler plate stuff. As a result, we have to again use *_start()* instead of *main()*. But additionally we also have to add a manual *_exit(0)* to terminate the code.  

## Difference between stdout and stderr
stdout is buffered while stderr isn't. What that means is, a printf to stdout may not occur immediately in the terminal, while a fprintf to stderr will. It is also possible to send printf output to a file, but stderr will always print on the terminal. So it is better for displaying critical errors.

## Type qualifiers
  + const, volatile, restrict are type qualifiers
  + const is better than #define in the sense that it can be treated like other variables - Mainly, its scope is restricted to the function it is inside
  + Volatile tells the compiler that variables value can change in ways that cannot be understood by looking at the code alone. It is useful if there a variable (global variable/semaphore etc.) can be changed by say an ISR or a different thread or a different processor core. For ex. if a global integer a=0; is defined at the top of a file and elsewhere there is a if/else where a==0 is done, without volatile qualifier, the compiler would do away with the if/else entirely. However, a context switch to a different thread or to an ISR may occur between the declaration and the if/else and a's value might change. This is where volatile is useful
  + restrict qualifier to a pointer tells the compiler that the object pointed to is not referenced by any other pointer - This may be useful to the compiler while it is doing optimization. But if, in the code, somehow aanother pointer points to the same object, compiler will *not* throw any error. 
  
## Storage classes
  + auto, extern, static, register are C storage classes
  + auto is the default. It is valid only for local variables. There is no difference between int a; and auto int a;
  + extern is for global variables defined outside in the file in question
  + register tells the compiler to *better* store the variable in a processor register rather than a RAM. But there is no guarantee compiler will do that. For ex., if the code uses the address of that variable (pointer operations), the compiler can't put the variable in a core register as it doesn't have an address. One important thing is that since a register can store only one word, one can use it for data types that have a width less than or equal to one word. 
  + static has two uses: It defines the scope of the variable/function to within a file it is defined in. When static is used inside a function for a local variable, it means its value won't be lost between function calls. static local variables are stored in .data (if initialized) or .bss (if uninitialized) just like global variables, but their scope is within the funciton in which they are defined.
  
## Embedded C
Into to [writing drivers](https://www.realdigital.org/doc/64f981fccbb1d8e03fd37b834bff6f6c)
[DMA transfer](https://www.oreilly.com/library/view/linux-device-drivers/0596000081/ch13s04.html)

## GCC stuff
  + gcc -Wall options displays all warnings
  + gcc -Wall -Werror converts warnings to errors
  + gcc -printsize option prints a table showing the sizes of .text, .data, .bss, .interrupts (IVT), .user_heap_stack areas.
  + Libraries for linking must be specified after the object file is specified. [Ref](https://stackoverflow.com/questions/9923495/undefined-reference-shm-open-already-add-lrt-flag-here). See how it affects linking:

### Makefile
Language agnostic. It runs some shell scripts based on target name supplied by the user. Ex. In 'makefile' (no extensions), we can have:

In [ ]:
CC=gcc
CFLAGS = -Wall -WError

all: hellomain.o hellofun.o
    $(CC) hellomain.o hellofun.o -o hello
 
hellomain.o: hellomain.c hellofun.h
    $(CC) $(CFLAGS) -c hellomain.c
 
hellofun.o: hellofun.c hellofun.h
    $(CC) $(CFLAGS) -c hellofun.c
 
clean:
    rm -rf *.o
    rm hello

When user does \\$make, the default target is chosen, which is the topmost target, i.e., *all*. Otherwise the user will have to specify, like \\$make clean. Based on the chosen target, make utility will look at dependencies of that target. So, for all, we need hellomain.o and hellofun.o to be present. These themselves are targets, so we need to basically execute them one by one. 

Here is another example:

In [ ]:
blah: blah.o
    gcc blah.o -o blah # Runs third

blah.o: blah.c
    gcc -c blah.c -o blah.o # Runs second

blah.c:
    echo "int main() { return 0; }" > blah.c # Runs first

In this example:
  + Make is given blah as the target, so it first searches for this target
  + blah requires blah.o, so make searches for the blah.o target
  + blah.o requires blah.c, so make searches for the blah.c target
  + blah.c has no dependencies, so the echo command is run
  + The cc -c command is then run, because all of the blah.o dependencies are finished
  + The top cc command is run, because all the blah dependencies are finished
  + That's it: blah is a compiled c program

# Linux processes
There are two types of processes: Foreground and background. Background processes dont need desktop environment (ex. gnome) or terminal to launch them. Ex. OS services. Foreground processes need to be launched from desktop env or terminal. Daemons are special background processes that start at system startup and never die (But user can control them via init process). 

A process can be created using system() method (old, has security issues) or fork(), exec() (newer, safer, more flexible). A process can be a parent or a child (launched by a parent). All processes will have PID (Process ID). and a PPID (Parent PID) with the exception of one process - The **Init** process. This is because the Init process is the mother of all processes. Its process ID is 1. In Ubuntu 20.04LTS, it is systemd (/sbin/init splash). 

A process can be in one of the following states: running, ready, waiting, stopped, zombie. A waiting process can be interruptible (by signals) or uninterruptible (waiting on hardware conditions). The fundamental way of controlling processes in Linux is by sending signals to them.

<img src = "ProcessState.png" width=600 />

Note that, in the beginning, the processes enter the ready queue when they have been loaded into the RAM, MMU configured etc. 

## IPC
There are many Inter-Process Communication methods in linux such as:
  + Shared files
  + Shared memory
  + Pipes (named or unnamed)
  + Message queues
  + Sockets 
  + Signals
  
A semaphore is used to avoid race conditions while accessing a shared memeory. A semaphore is like a counter where processes increment/decrement the counter. A mutex is a 1-bit semaphone and hence can be used to lock/unlock. Locking/Unlocking may require atomic operations (need to dig).

My understanding so far is that pipes are useful in a producer-consumer scenario between two processes, whereas shared memeory is useful where there are two or more processes want to both read from and write to the same resource (say, like a spreadsheet). Synchronization (blocking one process by another), if needed between the consumer and the producer in a pipe, mutex/semaphore can be used. For shared memory, a mutex/semaphore is almost always required. Named pipes are called FIFOs. Unnamed pipes are just called pipes [Ref](https://manpages.org/pipe/7). in C, pipes are created using pipe(), whereas a FIFO is created using mkfifo(). 

Sockets are basically same as networking sockets. Data is packetized and sent on a socket. Several processes in a local machine can receive on the socket, forward the data etc. 

### Resource locking
A spinlock is a lock that causes a thread trying to acquire it to simply wait in a loop ("spin") while repeatedly checking whether the lock is available. Since the thread remains active but is not performing a useful task, the use of such a lock is a kind of busy waiting. Once acquired, spinlocks will usually be held until they are explicitly released, although in some implementations they may be automatically released if the thread being waited on (the one that holds the lock) blocks or "goes to sleep".

### Signals
Signals can be sent by user to a program (Ex. Ctrl+C in a terminal sends the SIGINT signal to the running code for terminating it) or from one program to another (Ex. in C using kill(process_id, SIGINT) . It is a good idea for programmers to include code for handling signals, like the following

In [ ]:
// Catch all signals possible - it is vital we kill the DMA engine
// on process exit!
static void
setup_sighandlers(void)
{
    int i;
    for (i = 1; i < 32; i++) {
        // whitelist non-terminating signals
        if (i == SIGCHLD ||
            i == SIGCONT ||
            i == SIGTSTP ||
            i == SIGTTIN ||
            i == SIGTTOU ||
            i == SIGURG ||
            i == SIGWINCH ||
            i == SIGPIPE || // Python handles this
            i == SIGINT || // Python handles this
            i == SIGIO) {
            continue;
        }
        struct sigaction sa;
        memset(&sa, 0, sizeof(sa));
        sa.sa_handler = (void *) terminate;
        sigaction(i, &sa, NULL);
    }
}

## Blocking processes
  + [Blocking processes](https://linux.die.net/lkmpg/x1052.html)
  + [Blocking IO](https://www.oreilly.com/library/view/linux-device-drivers/0596000081/ch05s02.html#:~:text=Once%20the%20wait%20queue%20is,to%20sleep%20on%20this%20queue.)

## Fork, Exec, Wait, Exit
  + [Fork, exec, wait, exit](https://percona.community/blog/2021/01/04/fork-exec-wait-and-exit/)
  

## Debugging
  + strace - Shows system calls made by a code. Typical usage: *strace -ff ./myexecutable*
  + ltrace - Similar to strace, but for library calls.
  + lsof - lists open files/pipes etc., which user opened it, the command that opened it etc.  

## References
  + [Tecmint Linux process management](https://www.tecmint.com/linux-process-management/)
  + [Inter-process communication in Linux: Shared storage](https://opensource.com/article/19/4/interprocess-communication-linux-storage)
  + [When to use Pipes vs When to use Shared Memory](https://stackoverflow.com/questions/9701757/when-to-use-pipes-vs-when-to-use-shared-memory#:~:text=Pipes%3A%20Multiple%20process%20can%20Write,mutual%20exclusion%20for%20read%20%26%20write.)
  + [ps and ps aux commands quick intro](https://www.computernetworkingnotes.com/linux-tutorials/ps-aux-command-and-ps-command-explained.html)
  + [List of linux signals](https://www-uxsup.csx.cam.ac.uk/courses/moved.Building/signals.pdf)
  + [Handling signals in C](https://www.geeksforgeeks.org/signals-c-language/)

# Regex
The essential markups are:
  + () for grouping. Ex. (spider | bat)man to match spiderman or batman
  + {x,y} for looping. Ex. a{2,} - 'a' two or more times. c{4,6} - 'c' 4 to 6 times. 
  + [] for specifying range. Ex. [a-z] - small letters a to z. [0-9] digits 0 to 9
  + ^ for specifying beginning of a line. Ex. ^tty - lines that begin with 'tty'
  + \\$ for specifying end of the line. Ex. merica\$ - lines that end with 'merica'
  + ^ when used inside a range means NOT something. Ex. \[^a-f] - characters are *not* a, b, c, d, e, f. 
  + . wildcard - it will match anything other than a newline
  
The following are not essential. They can alternatively be specified using the above essential ones
  + \w same as \[_a-zA-z0-9\].
  + \d same as \[0-9\]
  + \+ repeated 1 or more times. Ex. a+ - 'a' repeated 1 or more times
  + \* repeated 0 or more times. 
  
Some commands that use regex (ex. grep), will require one to use forward slash to escape markups so they can be differentiated from actual characters. For ex. use \\. to mean the wildcard dot, whereas just a . would mean the actual character '.'. To confuse thing further, some commands require the exact opposite. You need to use trial and error to figure out. Some commands require that the regex be specified will a "" wrapper. 

Regex is very useful while doing grep. grep "tty\w\+" $(ls | grep "^s") means find lines that have 'tty' followed by a alphanumeric repeated 1 or more times in all files whose names start with an 's'
  

# Git
**Adding a repo from local computer**
  + In Github create an empty repo. Github will guide you on how to add a local repo from this point. So the following are just a repetition (it might be useful if you use a service that is different from Github)
  + Go into local directory and do *git init*
  + git add -A
  + git commit -m "whatever message"
  + git remote add origin https://github.com/saravanadinesh/repo_name.git
  + git branch -M main
  + git push -u origin main

## References
  1. [Basics of git](https://www.freecodecamp.org/news/learn-the-basics-of-git-in-under-10-minutes-da548267cc91/)

# Logic Analyzers
Used to probe 1s and 0s on I2C, SPI, UART etc., They sample the analog signal on the line and convert them to 1s or 0s. They do not show what is actually happening on the line like scopes do - the line may impulse noises here and there, which may be interpreted by the system it is hooked up to as 1s - the logic analyzer may miss them altogether. 

Logic analyzer specifications inlcude the number of channels, sampling frequency, how much buffer it has to store sampled data, can it accept external clock or not, does it have in-buit compression, is it real time (uploads data to say a PC monitor as soon as it get it, or store data in internal buffer and then once it is done, upload it to the computer). 

Just as with scopes, with logic analyzers, it is all about the analog front end: Noise, skews across channels, input capacitance, inductance, rise time, fall time, metastability issues etc. If you are pulling out a logic analyzer it means you are desperate. Either nothing is working, or everything is working 99% of the time, and you want to know why it is failing 1% of the time. There may be some tiny timing issue (often the logic analyzer is needed only while working with high speed protocols). Just the act of hooking up the logic analyzer will have an impact on the signal characteristics on a line. Costly analyzers have good AFEs and affect the lines less.  

Good logic analyzers have triggering facility where you can set a bit-string pattern and the analyzer will capture x bits of data that occurred before or after the trigger. It is basically like the digital version of a scope's trigger. Good logic analyzers also have low, high threshold levels (for deciding 0 vs. 1)

Some logic analyzers go one step beyond and parse the logic output and show human readable messages, highlight protocol errors and so on. 

### References
[1] https://www.circuitstoday.com/understanding-fpga-and-cpld
[2] https://numato.com/kb/cpld-vs-fpga-differences-one-use/ 